# <a id='toc1_'></a>[Quantum Chemistry Tutorial 1 - Modelling Basis](#toc0_)

1. **S. G. Nana Engo**, serge.nana-engo@facsciences-uy1.cm
    * Department of Physics, Faculty of Science, University of Yaounde I
2. **J-P. Tchapet Njafa**, jean-pierre.tchapet-njafa@univ-maroua.cm
    * Department of Physics, Faculty of Science, University of Maroua
       
April 2024

Dans cette série de tutoriels, nous allons aborder les notions essentielles chimie computationnelle ou chimie quantique. L'objectif principal étant d'apprendre et de travailler avec des méthodes et des logiciels informatiques de pointe en matière de chimie quantique. En plus des méthodes *ab-initio* traditionnelles, par exemple la théorie Hartree-Fock et post-Hartree-Fock, la théorie fonctionnelle de la densité (DFT) et la méthode semi-empirique xTB (Extended Tight Binding method) sont également discutés. Celles-ci sont appliquées par exemple à la prédiction de structures, à l'exploration de surfaces d'énergie potentielle. Il est à souligner le caractère unificateur de la chimie computationnelle : les mêmes méthodes théoriques servent d'outils pour résoudre divers problèmes dans tous les domaines de la chimie, de la chimie théorique et (in)organique et de la catalyse à la pharmaco- et biochimie.

A l'issue de cette série de tutoriels, l'étudiant doit être capable 
* d'expliquer certains concepts des méthodes modernes de chimie quantique : Hartree-Fock et la théorie fonctionnelle de la densité (DFT);
* d'expliquer certains aspects des logiciels de chimie quantique;
* de travailler avec des logiciels de chimie quantique de pointe, de prédire les structures et d'explorer les surfaces énergétiques potentielles;
* d'interpréter les résultats calculés en termes d'orbitales moléculaires et de modèles de structure électronique;
* d'utiliser un logiciel de chimie quantique pour résoudre des problèmes chimiques par voie informatique.


L'objectif de ce notebook (tutoriel 1) est de fournir les connaissances de base en chimie quantique nécessaire résoudre un problème de chimie avec HF et la DFT. De nombreux paramètres peuvent être modifiés, et certains ont un impact plus important que d’autres sur la précision et les ressources computationnelles. Cet ensemble de paramètres comprend le choix de l'état initial, le choix de l'ensemble de bases. De facon spécifique, à la fin de ce tutoriel, l'apprenant doit
* avoir compris les notions théoriques élémentaires de la structure électronique des molécules en seconde quantification;
* être capable d'utiliser le framework PySCF pour 
    * effectuer le calcul du champ moyen (SCF)
    * extraire
        * l'énergie totale
        * les index et les énergies du HOMO et du LUMO, 
        * le resumé des différentes énergies calculées
    * explorer le choix du set de bases


## <a id='toc1_2_'></a>[Structure électronique des molécules en première et seconde quantifications](#toc0_)

### <a id='toc1_2_1_'></a>[Hamiltonien d'une molécule en première quantification](#toc0_)

L'Hamiltonien d'une molécule constituée de noyaux $K$ et d'électrons $N$ est
\begin{equation*}
\mathtt{H}=\mathtt{T}_e+\mathtt{T}_n+\mathtt{U}_{en}+\mathtt{U}_{en}+\mathtt{U}_{nn},
\end{equation*}
où
- $\mathtt{T}_e= -\sum_i\frac{\hbar^2}{2m_e}\nabla^2_i$ est l'énergie cinétique électronique;
- $\mathtt{T}_n=-\sum_I\frac{\hbar^2}{2M_I}\nabla^2_I$ est l'énergie cinétique nucléaire;
- $\mathtt{U}_{en}= -\sum_{i,I}\frac{e^2}{4\pi\epsilon_0}\frac{Z_I}{|\mathbf{r}_i-\mathbf{R}_I|}$ est la répulsion Coulombienne entre les électrons et les noyaux;
- $\mathtt{U}_{ee}=+\frac{1}{2}\sum_{i\neq j}\frac{e^2}{4\pi\epsilon_0}\frac{1}{|\mathbf{r}_i-\mathbf{r}_j|}$ est la répulsion Coulombienne entre les électrons eux-mêmes;
- $\mathtt{U}_{nn}=+\frac{1}{2}\sum_{I\neq J}\frac{e^2}{4\pi\epsilon_0}\frac{Z_IZ_J}{|\mathbf{R}_I-\mathbf{R}_J|}$ est la répulsion Coulombienne entre les noyaux eux-mêmes.

$M_I$, $\mathbf{R}_I$ et $Z_I$ ($I=1,2$) désignent la masse, la position et le numéro atomique du *I*-ème noyau, et $\mathbf{r}_i$ est la position du *i*-ème électron. Par souci de concision, nous travaillons en unités atomiques, où l'unité de longueur est $a_0 = 1$ Bohr ($0.529167\times~10^{-10}$ m), l'unité de masse est la masse électronique $m_e$ et l'unité d'énergie est 1 Hartree ($1~\textrm{Hartree}~= ~{e^2}/{4\pi\epsilon_0 a_0}~=~27.2113~\textrm{eV}$). Dénotant $M_I'=M_I/m_e$, les différents termes de l'Hamiltonien moléculaire en unités atomiques s'écrit
\begin{equation*}
\begin{split}
\mathtt{T}_e&=-\sum_i\frac{\nabla^2_i}{2}, \qquad\mathtt{T}_n= -\sum_I\frac{\nabla^2_I}{2M_I'}, 
 \qquad\mathtt{U}_{en}= - \sum_{i,I}\frac{Z_I}{|\mathbf{r}_i-\mathbf{R}_I|},\\
\mathtt{U}_{ee}&=+\frac12\sum_{i\neq j}\frac{1}{|\mathbf{r}_i-\mathbf{r}_j|}
=+\sum_{i,j>i}\frac{1}{|\mathbf{r}_i-\mathbf{r}_j|},\\
\mathtt{U}_{nn}&=+\frac{1}{2}\sum_{I\neq J}\frac{Z_IZ_J}{|\mathbf{R}_I-\mathbf{R}_J|}
=+\sum_{I,J>I}\frac{Z_IZ_J}{|\mathbf{R}_I-\mathbf{R}_J|}.
\end{split}
\end{equation*}

### <a id='toc1_2_2_'></a>[Approximation de Born-Oppenheimer](#toc0_)

Nous nous intéressons principalement à la structure *électronique* de la molécule. Comme $M'\sim10^3$, nous appliquons l'**approximation de Born-Oppenheimer**, en traitant les noyaux comme des charges ponctuelles classiques. En conséquence, le terme d'énergie cinétique nucléaire $T_n$ est négligé, $\mathtt{U}_{nn}(\mathbf{R})$ est constant et l'Hamiltonien moléculaire est approché par un Hamiltonien électronique paramétré par $\mathbf{R}$,
\begin{equation*}
\begin{split}
\mathtt{H}(\mathbf{R}) &= \mathtt{U}_{nn}+\underset{\text{One-Electron Operators}}{\mathtt{T}_e + \mathtt{U}_{en}(\mathbf{R})} + \underset{\text{Two-Electron Operator}}{\mathtt{U}{ee}}\\
&= \mathtt{U}_{nn}+\underset{\text{One-Electron Operator}}{\sum_i h(i)} + \underset{\text{Two-Electron Operator}}{\sum_{i\ne j}v(i, j)}.
\end{split}
\end{equation*}

### <a id='toc1_2_3_'></a>[Energie potentiel de surface](#toc0_)

En vertu de l'équation de $\mathtt{H}(\mathbf{R})$ préceédente, l'équation de Schrödinger indépendante du temps non relativiste écrit
\begin{equation*}
\mathtt{H}(\mathbf{R})\psi_i(\mathbf{R},\mathbf{r})=E_i(\mathbf{R})\psi_i(\mathbf{R},\mathbf{r}),
\end{equation*}
où $\psi_i(\mathbf{R},\mathbf{r})$ désigne l'un des états propres et $E_i(\mathbf{R})$ correspond aux surfaces d'énergie potentielle moléculaire (PES), ce qui est important pour comprendre les processus chimiques tels que la formation et la rupture de liaison. En règle générale, la précision du calcul du PES (Potential Energy Surface) doit être limitée à la *précision chimique* $\epsilon=1 \textrm{kcal}\textrm{mol}^{-1}(\sim 1.59\times 10^{-3}\textrm{Hartrees},\, 43.3\textrm{meV})$.

L'énergie de l'état fondamental est donnée par
\begin{equation}\tag{1} 
E_0 = \frac{\langle\psi_0|\mathtt{H}|\psi_0\rangle}{\langle\psi_0|\psi_0\rangle},
\end{equation}
où $|\psi_0\rangle$ est l'état fondamental du système. Cependant, la dimensionnalité de ce problème croît exponentiellement avec le nombre de degrés de liberté. Pour résoudre ce problème, nous aimerions préparer $|\psi_0\rangle$ sur un calculateur quantique et mesurer directement la valeur moyenne Hamiltonienne (ou $E_0$). Alors, comment pouvons-nous faire cela concrètement?

Un bon point de départ pour résoudre ce problème est la méthode Hartree-Fock (HF), autrement nous allons prendre comme fonction d'état de réféfrence la fonction d'état HF. Cette méthode rapproche un problème à N corps en N problèmes à un corps où chaque électron évolue dans le champ moyen des autres.  Il est à noter que le dénominateur de l'Equation (1) n'est nécessaire que quand la fonction d'etat HF n'est pas normalisée au départ. Une facon d'améliorer cette fonction d'état HF est d'utiliser une combinaison linéaire des fonction d'état. Ce qui sera fait à travers les bases chimiques.


### <a id='toc1_2_4_'></a>[Déterminant Slater](#toc0_)

Un déterminant de Slater est un produit anti-symétrique d'un ou de plusieurs spin orbitals. Pour un système à $N$-electron, le déterminant de Slater s'écrit
\begin{equation*}
\begin{split}
\psi(\mathbf{r}_0, \mathbf{r}_1, \ldots, \mathbf{r}_{N-1})\equiv |\phi_{M-1},\cdots,  \phi_1, \phi_0 \rangle 
&=   \frac{1}{\sqrt{N!}}
\begin{vmatrix} \phi_0(\mathbf{r}_0) & \phi_1(\mathbf{r}_0) & \cdots & \phi_{M-1}(\mathbf{r}_0) \\
      \phi_0(\mathbf{r}_1) & \phi_1(\mathbf{r}_1) & \cdots & \phi_{M-1}(\mathbf{r}_1) \\
      \vdots & \vdots & \ddots & \vdots \\
      \phi_0(\mathbf{r}_{N-1}) & \phi_1(\mathbf{r}_{N-1}) & \cdots & \phi_{M-1}(\mathbf{r}_{N-1})
\end{vmatrix} \\
&\equiv |M-1,\dots, 1, 0 \rangle,
\end{split}
\end{equation*}
 L'échange des positions de deux électrons est équivalent à l'échange de deux rangées du déterminant de Slater, ce qui modifie le signe de la fonction d'état. Cela fournit la symétrie d'échange correcte pour la fonction d'état fermionique. Bien que le nombre de spin-orbitales considérées, $M$, soit généralement plus élevé que le nombre d'électrons dans le système, $N$, les électrons ne peuvent occuper que $N$ des spin-orbitales dans un déterminant Slater donné. En conséquence, le déterminant de Slater ne contient que les spin-orbitales occupées $N$.

### <a id='toc1_2_5_'></a>[Exemple d'illustration](#toc0_)

Pour rendre les correspondances plus claires, nous examinons comment une fonction d'état sera représentée pour un système quantique fictif. On considère les orbitales de spin $|A_\uparrow\rangle,\,|A_\downarrow\rangle,\,|B_\uparrow\rangle,\,|B_\downarrow\rangle$. Nous sommes libres de définir arbitrairement l'état Hartree-Fock de notre système fictif et de choisir les deux électrons dans l'orbitale $|A\rangle$. Nous sommes intéressés par la fonction d'état lorsque la composante $z$ du spin est nulle.

Labellisons chaque orbitale ainsi qu'il suit :
\begin{align*}
 &|A_\uparrow\rangle=|00\rangle=|\mathbf{0}\rangle,  &&|A_\downarrow\rangle=|01\rangle=|\mathbf{1}\rangle,
&&|B_\uparrow\rangle=|10\rangle=|\mathbf{2}\rangle,  &|B_\downarrow\rangle=|11\rangle=|\mathbf{3}\rangle.
\end{align*}
L'état de Hartree-Fock possède les deux électrons dans les orbitales $|A\rangle$. Un état HF incorrectement symétrisé serait donc $|A_\uparrow\rangle_1|A_\downarrow\rangle_2 = |\mathbf{0}\rangle_1|\mathbf{1}\rangle_2$, où les indices indiquent l'électron décrit par chaque orbitale. La fonction d'état HF correctement antisymétrisée serait 
\begin{equation*}
\begin{split}
    |\Psi_{\mathrm{HF}}\rangle =  \frac{1}{\sqrt{2}}
		\begin{vmatrix}
		A_\uparrow(\mathbf{x}_0) & A_\downarrow(\mathbf{x}_0)\\
		A_\uparrow(\mathbf{x}_1) & A_\downarrow(\mathbf{x}_1)\\
		\end{vmatrix}
		&= \frac{1}{\sqrt{2}}  \Big( A_\uparrow(\mathbf{x_0}) A_\downarrow(\mathbf{x_1}) - A_\downarrow(\mathbf{x_0}) A_\uparrow(\mathbf{x_1})  \Big)\\
&=\frac{1}{\sqrt{2}} (|\mathbf{0}\rangle_1|\mathbf{1}\rangle_2 - |\mathbf{1}\rangle_1|\mathbf{0}\rangle_2 ).
\end{split}
\end{equation*}
Si nous considérons maintenant les excitations au-dessus de l'état HF, alors une fonction d'état générale avec $s_z=0$ qui a été correctement antisymétrisée est
donnée par
\begin{equation*}
\begin{aligned}
|\Psi\rangle =& \frac{\alpha}{\sqrt{2}}(|\mathbf{0}\rangle_1|\mathbf{1}\rangle_2 - |\mathbf{1}\rangle_1|\mathbf{0}\rangle_2 ) 
+\frac{\beta}{\sqrt{2}} (|\mathbf{2}\rangle_1|\mathbf{3}\rangle_2 - |\mathbf{3}\rangle_1|\mathbf{2}\rangle_2 ) \\
+& \frac{\gamma}{\sqrt{2}} (|\mathbf{0}\rangle_1|\mathbf{3}\rangle_2 - |\mathbf{3}\rangle_1|\mathbf{0}\rangle_2 ) + \frac{\delta}{\sqrt{2}} (|\mathbf{1}\rangle_1|\mathbf{2}\rangle_2 - |\mathbf{2}\rangle_1|\mathbf{1}\rangle_2 ).
\end{aligned}
\end{equation*}
Comme nous avons $N=2$ électrons et $M=4$ spin-orbitales, nous pouvons voir que nous avons seulement besoin de $N \lceil\mathrm{log_2}(M)\rceil = 2 \times \lceil\mathrm{log_2}(4)\rceil = 4$ qubits pour stocker la fonction d'état.


## <a id='toc1_3_'></a>[Seconde quantification - Construction d'un opérateur Hamiltonien fermionique](#toc0_)

### <a id='toc1_3_1_'></a>[Génération des intégrales moléculaires](#toc0_)

L'Hamiltonien  est exprimé dans la base des solutions de la méthode HF, également appelées Orbitales Moléculaires (OM) :
$$
\mathtt{H}_{elec}=\sum_{pq} h_{pq} a^{\dagger}_p a_q + 
\frac{1}{2} \sum_{pqrs} h_{pqrs}  a^{\dagger}_p a^{\dagger}_q a_r  a_s
$$
avec 
* les **intégrales à 1 électron**
$$
h_{pq} = \int \phi^*_p(r) \left( -\frac{1}{2} \nabla^2 - \sum_{I} \frac{Z_I}{|\mathbf{r}_i-\mathbf{R}_I|} \right)   \phi_q(r)d\mathbf{r}
$$
qui décrivent l’énergie cinétique des électrons individuels et leurs interactions Coulombiennes avec les champs électriques du noyau ;
* et **intégrales à 2 électrons**
$$
h_{pqrs} = \int \frac{\phi^*_p(r_1)  \phi^*_q(r_2) \phi_r(r_2)  \phi_s(r_1)}{|\mathbf{r}_1-\mathbf{r}_2|}d\mathbf{r}_1d\mathbf{r}_2,
$$
décrivent les interactions Coulombiennes entre les électrons;
* $a^\dagger_p$ l'**opérateur de création** d'un électron sur le spin-orbite $p$;
* $a_q$ l'**opérateur d'annihilation** d'un électron sur sur le spin-orbite $q$;
* $a^{\dagger}_p a_q $ est l'**opérateur d'excitation**, qui excite un électron du spin-orbite occupé $\phi_q$ vers le spin-orbite inoccupé $\phi_p$.

Les MO ($\phi_u$) peuvent être occupés ou virtuels (inoccupés). Un MO peut contenir 2 électrons. Ces MO sont les solutions de la méthode HF, où chaque l´electron est soumis au champ moyen créé par les autres électrons. Cependant, dans ce qui suit, nous travaillons en fait avec des orbitales de spin qui sont associées à un spin up ($\alpha$) d'électron spin down ($\beta$). Ces deux spins sont également communément désignés par $\alpha$ et $\beta$, respectivement. Ainsi, les orbitales de spin peuvent contenir un électron ou être inoccupées.

Dans le tableau ci-dessous resume les correspondances première-seconde quantification des divers éléments du Hamiltonien moléculaire. 
$$
\begin{array}{|l|l|l|}\hline
\textbf{Approximation BO}  &  \textbf{Combinaison en 2e quantification} &  \textbf{Description}
\\\hline
-\frac12\sum_i\nabla^2_i & -\frac12\sum_{p,q}\langle p|\nabla^2_p|q\rangle a^\dagger_p a_q & \mathtt{T}_e \\\hline
 -\sum_{i,I}\frac{Z_I}{|\mathbf{r}_i-\mathbf{R}_I|} &
 -\sum_{p,q} \langle p|\frac{Z_I}{|\mathbf{r}_p-\mathbf{R}_I|}|q\rangle a^\dagger_p a_q
 & \mathtt{U}_{en}\\\hline
\sum_{i,j>i}\frac{1}{|\mathbf{r}_i-\mathbf{r}_j|} & 
\sum_{p,q,r,s}\langle pq \big|\frac{1}{|\mathbf{r}_i-\mathbf{r}_j|}\big|rs\rangle a^\dagger_p a^\dagger_q a_r a_s & 
 \mathtt{U}_{ee}\\\hline
\end{array}
$$

*Un des avantages apporté par le formalisme de seconde quantification est que la propriété d'anti-symétrie des fonctions d'état pour l'échange de fermions identiques, prise en compte manuellement dans une approche de première quantification en considérant les déterminants de Slater, est automatiquement imposée par le relations d'anti-commutation des opérateurs de création et d'annihilation. Cependant, le prix à payer est de travailler avec un nombre non fixe de particules, qui reste de toute façon une quantité conservée.*

### <a id='toc1_3_2_'></a>[Répresentation nombre d'occupation](#toc0_)

En seconde quantification, le déterminant de Slater est représenté par un **vecteur nombre d'occupation**
\begin{equation*}
|f\rangle =|f_{M-1},\dots,f_{i-1},f_i,f_{i+1},\dots, f_0\rangle, 
\end{equation*}
avec
- $f_i=1$ lorsque le spin-orbital $\phi_i$ est occupé et donc présent dans le déterminant de Slater;
- $f_i = 0$ lorsque le $\phi_i$ spin-orbital est vide et donc absent du déterminant de Slater.

- Le vecteur $|f_i\rangle$ est appelé **vecteur nombre d'occupation de l'orbite fermionique i**, parce que
\begin{align*}
& |0\rangle=\begin{pmatrix}1\\0\end{pmatrix}:\text{état vide}, & |1\rangle=\begin{pmatrix}0\\1\end{pmatrix}:\text{état occupé}.
\end{align*}

Dans la deuxième représentation de quantification, au lieu de poser la question *Quel électron est dans quel état?*, nous posons la question *combien de particules y a-t-il dans chaque état?*.

### <a id='toc1_3_3_'></a>[Opérateurs fermioniques](#toc0_)

Les opérateurs fermioniques $a^\dagger_i$ et $a_i$ obéissent aux relations d'anticommutation fermioniques suivantes:
\begin{align*}
&\{a_p,a^\dagger_q\} = a_pa^\dagger_q + a^\dagger_q a_p = \delta_{pq}, 
&\{a_p, a_q \} = \{a^\dagger_p, a^\dagger_q\} = 0 .
\end{align*}
La nature fermionique des électrons implique que les fonctions d'état à plusieurs électrons doivent être antisymétriques par rapport à l'échange de particules. Cela se reflète dans la manière dont les opérateurs de création fermionique et d'annhilation agissent sur les déterminants $|f\rangle$:
\begin{equation*}
\begin{aligned}
&a_i|f_{M-1},\dots,f_{i-1},f_i,f_{i+1},\dots, f_0\rangle = \delta_{f_i,1}
(-1)^{\sum_{m<i}f_m}|f_{M-1},\dots,f_{i-1},f_i\oplus 1,f_{i+1},\dots, f_0\rangle,\\
&a^\dagger_i|f_{M-1},\dots,f_{i-1},f_i,f_{i+1},\dots, f_0\rangle = \delta_{f_i,0}
(-1)^{\sum_{m<i}f_m}|f_{M-1},\dots,f_{i-1},f_i\oplus 1,f_{i+1},\dots, f_0\rangle`,  \\
&a|0\rangle=a^\dagger|1\rangle=0.
\end{aligned}
\end{equation*}
- Le terme de phase $p_i=(-1)^{\sum_{m<i}f_m}$ désigne la parité qu'impose l'anti-symétrie d'échange des fermions.
    - $p_i=-1$ si le nombre d'électrons est impair dans ces orbitales de spin,
    - $p_i= 1$ si le nombre d'électrons est pair dans ces orbitales de spin.

- Le symbole $\oplus$ représente l'addition modulo 2, i.e.,  $1 \oplus 1=0 $ et $0 \oplus 1=1$.

- L'opérateur de création $a^\dagger_i$ met un électron dans une orbitale inoccupée *i* ou dans $|f_i\rangle$.

- L'opérateur d'annihilation $a_i$ supprime un électron dans une orbitale occupée *i* ou en $|f_i\rangle$.

- $a_i^2 =(a^\dagger_i)^2= 0$ pour tout $i$. On ne peut pas créer ou anéantir un fermion dans le même mode deux fois.

Par example,
\begin{align*} 
&a^\dagger_1 |0\rangle_1 = |1\rangle_1,&&a^\dagger_1 |1\rangle_1 =(a^\dagger_i)^2|0\rangle_1= 0,&& a_1 |1\rangle_1 =|0\rangle_1,&a_1 |0\rangle_1 =a_1^2|1\rangle_1=0. 
\end{align*}
Notez qu'ici $a^\dagger_1|1\rangle_1=0$ et $a_1 |0\rangle_1=0$ signifie le vecteur zéro et non $|0\rangle_1$. 

En utilisant également de tels opérateurs, nous pouvons exprimer
\begin{equation*}
|0\rangle |1\rangle |1\rangle |0\rangle = a^\dagger_1a^\dagger_2 |0\rangle^{\otimes 4}.
\end{equation*}

 L'opérateur d'occupation orbitale est donné par
\begin{align*}
&n_i = a^\dagger_i a_i, &n_i |f_{M-1},\dots,f_i,\dots, f_0\rangle= f_i |f_{M-1},\dots,f_i,\dots, f_0\rangle,
\end{align*}
et il compte le nombre d'électrons dans une orbitale donnée, c'est-à-dire
\begin{align*} 
&n_i |0\rangle_i = 0, &n_i |1\rangle_i = |1\rangle_i. 
\end{align*}

### <a id='toc1_3_4_'></a>[Exercise](#toc0_)

In **Exemple d'illustration** above, we have considered a fictitious system described by spin orbitals $|A_\uparrow\rangle,\,|A_\downarrow\rangle,\,|B_\uparrow\rangle,\,|B_\downarrow\rangle$ labelled as
\begin{align*}
 &|A_\uparrow\rangle=|00\rangle=|\mathbf{0}\rangle,  &&|A_\downarrow\rangle=|01\rangle=|\mathbf{1}\rangle,
&&|B_\uparrow\rangle=|10\rangle=|\mathbf{2}\rangle,  &|B_\downarrow\rangle=|11\rangle=|\mathbf{3}\rangle.
\end{align*}
We assume that the Hartree-Fock state for this system **has both electrons occupying the $|A\rangle$ orbitals**. We store the occupations of the spin orbitals $|A_\uparrow\rangle,\,|A_\downarrow\rangle,\,|B_\uparrow\rangle,\,|B_\downarrow\rangle$ which we order as 
$|f_{B_\downarrow},f_{B_\uparrow},f_{A_\downarrow},f_{A_\uparrow}\rangle$, with $f_i=0,1$ (in the fermionic Fock space).

1. Write down, in the fermionic Fock space, the Hartree-Fock state $|\Psi_{\rm HF}\rangle$, that is, the corresponding of the antisymmetrized Slater deter-
minant obtained in **Exemple d'illustration**.
$$
|\Psi_{\rm HF}\rangle = |0011\rangle .
$$
1. Write down, in the fermionic Fock space, the $s_z = 0$ state function.
$$
|\Psi_{\rm HF}\rangle = \alpha|0011\rangle + \beta|1100\rangle + |1001\rangle + |0101\rangle .
$$


## Calcul du champ moyen ou mean-field

Le calcul du champ moyen est généralement la première étape de toutes les approches post Hartree-Fock. Les approches en champ moyen résolvent l'équation de Schrödinger en faisant la moyenne du terme de répulsion électron-électron de manière auto-cohérente ou SCF. En d’autres termes, chaque électron n’interagit pas explicitement avec les autres électrons.

Les trois méthodes HF communément utilisées sont (voir la figure ci-dessous) :

![HF_Orb.png](./Graphics/HF_Orb.png)

1. **RHF (Restricted Hartree – Fock)** utiliśee pour des molécules à couches pleines ou fermées. Les spin-orbitales sont soit $\alpha$, soit $\beta$ et tous les orbitales sont doublement occupés par des spin-orbitales $\alpha$ et $\beta$;</br> 
2. **ROHF (Restricted Open-Shell Hartree–Fock)** utilisée pour des molécules à couches ouvertes où le nombre d'électrons les orbitales n'est pas le même. ROHF utilise autant que possible les orbitales doublement occupées et les orbitales une fois occupées par les électrons non-apariés;
3. **UHF (Unrestricted Hartree-Fock)** utilisée pour des molécules à couches ouvertes où le nombre d'électrons les orbitales n'est pas le même. Les orbitales UHF peuvent avoir avoir des spin $\alpha$ ou $\beta$, mais les orbitales $\alpha$ et $\beta$ peuvent avoir des composants spatiales différents.

Les équivalents DFT sont,

4. **RKS (Kohn-Sham restreint)**,
6. **ROKS (Restricted Open-Shell Kohn – Sham)**,
5. **UKS (Kohn-Sham sans restriction)**.

### PySCF

![Pyscf.png](./Graphics/Pyscf.png)

**Python-based Simulations of Chemistry Framework (PySCF)** est un programme de chimie computationnelle *ab initio* implémenté nativement dans le langage de programme Python. Le package vise à fournir une plate-forme simple, légère et efficace pour le développement et le calcul de codes de chimie quantique. Il fournit diverses fonctions pour faire, au niveau non relativiste, 
* la théorie de Hartree-Fock,
* la Second-order Møller–Plesset perturbation theory (MP2),
* la théorie fonctionnelle de la densité (DFT),
* la Multi-configuration self-consistent field (MCSCF),
* la théorie des clusters couplés (CC).

PySCF est utilisé dans la plupart des packages de calculs quantiques pour les calculs HF (première quantification) et diverses étapes des transformations de seconde quantification.

Il est à noter que la méthode SCF ou Self Consistent Field est une méthode variationnelle où la fonction d'essai est un déterminant de Slater ou une combinaison linéaire de ces déterminants. 

In [ ]:
import pyscf

pyscf.__version__

### <a id='toc1_4_1_'></a>[Information sur la molécule - Création de l'objet molecule avec `pyscf.gto.Mole()`](#toc0_)

Lors de la définition d'une molécule, on spécifie les propriétés moléculaires. 
* L’une d’elles est la charge totale du système, c’est-à-dire la différence entre le nombre d’électrons et la charge nucléaire totale (la valeur par défaut est 0). 
* Une autre quantité importante à spécifier est le spin, défini par la différence entre le nombre d'électrons alpha et bêta (la valeur par défaut est 0). Cela permet de modéliser des systèmes radicalaires : un spin de 0 caractérise une molécule à couche restreinte tandis qu'un spin différent de 0 définit un système à couche ouverte.

Considérons le dimère d'eau (deux molécules d'eau à liaison hydrogène) illustré par la figure suivante

<!-- ![water_dimer.jpg](attachment:water_dimer.jpg) -->
![water_dimer.jpg](./Graphics/water_dimer.jpg)


* Importer les modules nécessaires

In [ ]:
from pyscf import gto # Gaussian type orbitals
from pathlib import Path # For filesystem paths
# For 3D plotting
try :
    import py3Dmol
except:
    !pip install py3Dmol
    import py3Dmol
py3Dmol.__version__

* Définir la PySCF molecule object 

In [ ]:
mol_xyz = """
    O  -1.551007  -0.114520   0.000000
    H  -1.934259   0.762503   0.000000
    H  -0.599677   0.040712   0.000000
    O   1.350625   0.111469   0.000000
    H   1.680398  -0.373741  -0.758561
    H   1.680398  -0.373741   0.758561""" # Ref: https://cccbdb.nist.gov/expgeom1x.asp

smi_key = 'Dwater'

# Verify if the "{smi_key}" subdirectory exist and if not, create it
working_dir = Path(f'./{smi_key}')
# Create the output directory if it doesn't exist
working_dir.mkdir(parents=True, exist_ok=True)
# Create the pyscf log file
path_pyscf_log = working_dir / f'{smi_key}_pyscf.txt'

# PySCF molecule object
DWat_mol=gto.Mole(
    atom=mol_xyz,
    basis='def2-TZVP',
    charge=0,      # 0 by default
    spin=0,        # 0 by default, defined as (n_up - n_down == nelec_alpha - nelec_beta)
    unit='Angstrom', # Can also be 'Bohr'
    output = path_pyscf_log # to store some output calculations
)
DWat_mol.build()

# # 3D representation
xyz_view = py3Dmol.view(width=300,height=300)
xyz_view.addModel(DWat_mol.tostring(format="xyz"),'xyz')
xyz_view.setStyle({'stick': {}, 'sphere': {'scale': .30}})
xyz_view.zoomTo()
xyz_view.show()


In [ ]:
print(f'Le nombre total d\'électrons est {DWat_mol.nelectron} \
et le nombre total d\'électrons (alpha, béta) est {DWat_mol.nelec}\n')
print(f'Les index (0-Based) du (HOMO,LUMO) sont {DWat_mol.nelectron//2 -1, DWat_mol.nelectron//2}\n')
print(f'Le nombre d\'orbitales atomiques, dans la base {DWat_mol.basis}, est {DWat_mol.nao}\n')
print(f'L\'énergie nucléaire vaut {DWat_mol.energy_nuc()} Hartrees')

### <a id='toc1_4_2_'></a>[Création d'un objet mean-field (mf) avec `scf.RHF()`](#toc0_)

Pour la plupart des usages, les types de calculs SCF pertinents seront :

1. RHF (Restricted Hartree – Fock)
2. UHF (Hartree-Fock sans restriction)
3. ROHF (Restricted Open-Shell Hartree–Fock)


L'initialisation de l'objet champ moyen pour ces six types de calculs est illustrée ci-dessous. Ces classes nécessitent au moins un argument, à savoir un objet molécule, mol.

1. `mf=scf.RHF(mol)`
2. `mf=scf.UHF(mol)`
3. `mf=scf.ROHF(mol)` (où mol est définie comme étant à couche ouverte (open-shell), mol.spin!=0)
4. `mf=scf.RKS(mol)`
5. `mf=scf.UKS(mol)`
6. `mf=scf.ROKS(mol)` (où mol est définie comme open-shell, mol.spin!=0)

La première étape consiste à créer l'objet mean-field (RHF dans ce cas)

In [ ]:
from pyscf import scf

mf=scf.RHF(DWat_mol)

Par la suite, on exécute le caclul avec la méthode `.run()`ou `.kernel()`.

In [ ]:
mf.kernel()

Une fois l'exécution du noyau (kernel) SCF achevée, l'objet mean-field (mf) est mis à jour avec plusieurs attributs de sortie utiles :
1. `mf.mo_occ` - Occupation MO (vecteur de longueur égale au nombre de MO)

In [ ]:
mf.mo_occ

In [ ]:
mf.get_occ()

In [ ]:
import plotly.express as px

# Plot the MO Occupations
fig = px.line(y=mf.mo_occ, markers=True, title="Molecular Orbital (MO) Occupations")
fig.update_layout(xaxis_title="Orbital Index (0-Based)", yaxis_title="MO Occupation")
fig.show()

On note, sachant que l'indexation commence à 0, que 
* l'orbite moléculaire la plus occupée (HOMO) a pour indexe 9 (nombres électrons//2 - 1),
* et de l'orbite moléculaire la plus basse inoccupée (LUMO) a pour indexe 10 (nombres électrons//2).

Pour avoir les indexes du LUMO et l'HOMO à partir du mf:

In [ ]:
lumo_idx = mf.mo_occ.tolist().index(0.)
homo_idx = lumo_idx - 1
print(f'Les index (0-Based) du (HOMO,LUMO) sont {homo_idx,lumo_idx}')

2. Energies Homo, Lumo et gap en eV

$$E_g = E_{\rm LUMO} - E_{\rm HOMO}$$

In [ ]:
from pyscf.data.nist import HARTREE2EV as au2ev 

print(f'Energie du Homo = {mf.mo_energy[homo_idx] * au2ev} eV')
print(f'Energie de Lumo = {mf.mo_energy[lumo_idx] * au2ev} eV')
print(f'Energie du gap Homo-Lumo = {abs(mf.mo_energy[homo_idx] - mf.mo_energy[lumo_idx]) * au2ev} eV')

In [ ]:
# Plot the MO Energies (i.e. eigenvalues of the Fock matrix)
fig = px.line(y=mf.mo_energy, markers=True, title="Molecular Orbital (MO) Energies (a.u.)")
fig.update_layout(xaxis_title="Orbital Index (0-Based)", yaxis_title="MO Energies (a.u.)")
fig.show()

2. `mf.e_tot` - Énergie SCF totale en unités de Hartrees

In [ ]:
mf.e_tot

2. `mf.dump_scf_summary()` - Synthèse des du calcul SCF effectué.

In [ ]:
mf.dump_scf_summary()

In [ ]:
mf.scf_summary

On peut aussi spécifier les paramètres de calculs du mf.

In [ ]:
mf=scf.RHF(DWat_mol)

mf.conv_tol=1e-12 # the difference in the SCF energy (in Hartrees) between two successive cycles
mf.conv_tol_grad=1e-8 # the root-mean-square of the orbital gradient
mf.direct_scf_tol=1e-13 #the threshold for discarding integrals
mf.init_guess='atom' #  vital for the efficient completion of the SCF procedure
mf.max_cycle=100 # the maximum number of SCF cycles that should be performed before the calculation terminates. For systems that are notoriously difficult to converge, the value should be increased to 100 or even 1000
mf.max_memory=4000 #determines the maximum amount of memory (in Megabytes) that PySCF is allowed to utilize during the SCF procedure
mf.verbose=5 #controls the print level for the mean-field object (in the 'Dwat_HF.txt' defined above)

mf.kernel()

### <a id='toc1_4_3_'></a>[Création d'un objet mean-field (mf) avec `scf.RKS()`](#toc0_)

La méthode SCF la plus largement utilisée en chimie quantique est sans aucun doute la théorie de la fonctionnelle de la densité (DFT), et exécuter un calcul DFT dans PySCF est presque aussi simple que d'exécuter un calcul Hartree-Fock.

Afin d'accéder à des fonctions supplémentaires qui ne relèvent pas de la norme SCF/HF, il est nécessaire d'importer le module dft (théorie fonctionnelle de la densité).

In [ ]:
from pyscf import dft

mfd=dft.KS(DWat_mol)

mfd.conv_tol=1e-12
mfd.conv_tol_grad=1e-8
mfd.direct_scf_tol=1e-13
mfd.init_guess='1e'
mfd.max_cycle=100
mfd.max_memory=8000

mfd.xc='wB97X-D' # la fonctionnelle
mfd.grids.atom_grid=(50,194)
mfd.grids.becke_scheme=dft.gen_grid.original_becke
mfd.grids.prune=None
mfd.grids.radi_method=dft.radi.gauss_chebyshev
mfd.grids.radii_adjust=None
mfd.grids.verbose=0
mfd.small_rho_cutoff=1e-10

mfd.kernel()

In [ ]:
mfd.dump_scf_summary()

In [ ]:
mfd.scf_summary

RHF et RKS (ainsi que UHF et UKS) partagent de nombreux attributs, nous ne couvrirons donc que ceux qui concernent spécifiquement DFT. Ces attributs supplémentaires concernent principalement la fonctionnelle d'échange-corrélation et le maillage utilisé pour son intégration numérique.

#### <a id='toc1_4_3_1_'></a>[Attribut xc](#toc0_)

(NO DEFAULT)

L'attribut xc définit l'approximation de la fonctionnelle de la densité. Il vise à absorber la corrélation extrêmement complexe de nombreuses particules et les effets fermioniques (d'échange) dans une énergie et un potentiel électroniques de champ moyen apparemment simples mais formellement exacts et théoriquement existants.

Un grand nombre de fonctionnalles de corrélation-échange sont disponibles pour une utilisation dans PySCF, essentiellement toutes celles qui sont disponibles dans la version installée de libxc.

![DFT_Accuracy.jpg](./Graphics/DFT_Accuracy.jpg)

<!-- <center><img src="Graphics/DFT_Accuracy.jpg" width="450"/></center> -->

Les règles d'analyse de l'attribut xc soient expliquées dans le [manuel PySCF](http://sunqm.github.io/pyscf/dft.html#customizing-xc-functional). Nous allons passer en revue les principaux exemples pertinents ici :

1. Fonctionnalité de corrélation-échange standard (précodée)

   * Un exemple classique de ce scénario est la très populaire fonction `B3LYP`. Pour exécuter un calcul B3LYP, spécifiez simplement `mf.xc='B3LYP'`.
  
   * Tant que les virgules et les opérateurs ne sont pas utilisés, PySCF supposera que la chaîne indique une fonction de corrélation-échange complète. D'autres exemples peuvent inclure `mf.xc='PBE'`, `mf.xc='TPSS'`, `mf.xc='M06-L'`, etc.

2. Combinaison d'une fonction d'échange unique avec une fonction de corrélation unique

   * Il s'agit d'un choix populaire, en particulier pour l'utilisation de diverses combinaisons des anciennes fonctionnelles d'échange et de corrélation des années 1980 et 1990.
  
   * Dans PySCF, l'insertion d'une virgule dans la chaîne d'attribut xc indique une séparation entre l'échange (à gauche de la virgule) et la corrélation (à droite de la virgule). Par exemple, si on veut exécuter un échange B88 avec une corrélation PBE, on indique `mf.xc='B88,PBE'`.
  
   * Étant donné que la fonctionnelle de corrélation-échange PBE est une combinaison d'échange PBE et de corrélation PBE, il s'ensuit que `mf.xc='PBE'` et `mf.xc='PBE,PBE'` sont des façons exactement équivalentes d'utiliser la fonctionnelle PBE.
  
   * D'autres combinaisons populaires d'une fonction d'échange unique avec une fonction de corrélation unique incluent `mf.xc='B88,LYP'`, `mf.xc='revPBE,PBE'` et `mf.xc='OPTX,LYP' `. Ces combinaisons sont historiquement connues sous le nom de BLYP, revPBE et OLYP, et peuvent être appelées de manière équivalente `mf.xc='BLYP'`, `mf.xc='revPBE'` et `mf.xc='OLYP'`.
  
   * Cette option est particulièrement utile lorsque l'on s'intéresse à une combinaison non conventionnelle, par exemple, la combinaison d'échange PBE avec corrélation TPSS, `mf.xc='PBE,TPSS'`.

3. Fonction d'échange unique ou fonction de corrélation unique

   * L'utilisation d'une fonctionnelle d'échange unique, par exemple, l'échange PBE uniquement, est aussi simple que `mf.xc='PBE,'`. De même, la corrélation PBE ne peut être spécifiée qu'avec `mf.xc=',PBE'`. Dans le premier cas (échange PBE uniquement), le fait qu'il y ait une virgule indique à l'analyseur syntaxique qu'une fonctionnelle xc précodée n'est pas utilisée. Par conséquent, tout ce qui se trouve à gauche de la virgule (PBE) est considéré comme la fonctionnelle d'échange, tandis que tout ce qui se trouve à droite de la virgule (rien) est considéré comme la fonctionnelle de corrélation. La situation inverse s'applique au deuxième cas avec corrélation PBE uniquement.

4. Fonctionnalité de corrélation-échange personnalisée

   * Naturellement, il est possible de personnaliser entièrement l'attribut xc. Encore une fois, tout ce qui se trouve à gauche de la virgule est supposé faire référence à l'échange, et tout ce qui se trouve à droite de la virgule est supposé faire référence à la corrélation.
  
   * Afin de commencer avec quelque chose de simple, nous montrons comment spécifier la fonctionnelle xc [PBE0](https://aip.scitation.org/doi/abs/10.1063/1.478522) en la construisant à partir de ses composants (la fonctionnelle facile s'écrit simplement `mf.xc='PBE0'`). Le PBE0 comprend 25 % d'échange exact (Hartree-Fock), 75 % d'échange PBE et 100 % de corrélation PBE : `mf.xc='0.25*HF+0.75*PBE,PBE'`.
  
   * Un exemple plus compliqué est celui de B3LYP. B3LYP comprend 20 % d'échange exact (Hartree-Fock), 8 % d'échange LDA, 72 % d'échange B88, 19 % de corrélation VWN et 81 % de corrélation LYP. Malheureusement, il existe 6 paramétrages de la fonctionnelle de corrélation VWN LDA, ce qui a conduit à beaucoup de confusion au fil des ans.
  
   * PySCF, TURBOMOLE, GAMESS, ORCA et PSI4 utilisent tous le paramétrage VWN5, et les deux affectations suivantes sont équivalentes :
     * `mf.xc='B3LYP'`
     * `mf.xc='0.2*HF+0.08*LDA+0.72*B88,0.19*VWN5+0.81*LYP'`
   * Q-Chem et NWChem utilisent le paramétrage VWN1RPA :
     * `mf.xc='0.2*HF+0.08*LDA+0.72*B88,0.19*VWN_RPA+0.81*LYP'`
   * Gaussian utilise la paramétrisation VWN3 :
     * `mf.xc='0.2*HF+0.08*LDA+0.72*B88,0.19*VWN3+0.81*LYP'`    
  

### Analyse du champ moyen

On utilise la méthode `analyze` de l'objet de champ moyen pour imprimer les propriétés moléculaires pertinentes. 

In [ ]:
mf.analyze(verbose = 4) # store in 'Dwat_HF.txt'

## Choix du set de bases

L'équation de Schrödinger à plusieurs électrons est généralement résolue sur une base d'orbitales moléculaires exprimées sous la forme d'une **combinaison linéaire de ses orbitales atomiques (LCAO, Linear combination of atomic orbitals)**, 
$$
     \phi_a(\mathbf{r}) =
     \sum_A^\mathrm{atomes}
     \sum_\alpha C^A_{\alpha a}
     \chi^A_{\alpha}(\mathbf{r} - \mathbf{R}_A)
$$
Les coefficients LCAO, utilisés pour construire des orbitales moléculaires (MO), sont obtenus grâce à la minimisation de l'énergie totale, selon le principe variationnel. Pour plus de commodité, la base est généralement représentée par des fonctions centrées sur les atomes (les positions atomiques) pour les systèmes moléculaires, ou par des fonctions d'état plane en physique du solide.

### Ensembles de bases traditionnels

Les deux classes traditionnelles  d'orbitales de base approximatives couramment utilisées sont :
* les **Slater-type orbitals (STOs)** basées sur le déterminant de Slater, construitent avec des fonctions exponentielles décroissantes $e^{-\alpha r}$,
* et les **orbitales cartésiennes de type Gaussien (GTO)**, construitent avec des fonctions Gaussiennes $e^{-\alpha r^2}$. L’utilisation de ces fonctions Gaussiennes est bien plus efficace que la stratégie STO pour calculer les nombreuses intégrales de chevauchement nécessaires à la chimie quantique.

Ces deux types de fonctions de base peut être combiné comme **STO-nG (Slater-type orbital-n Gaussians)**, où n est le nombre de Gaussiennes utilisées pour faire les approximations.

Les ensembles de base sont construits à partir de nombreuses primitives Gaussiennes. La notation de Pople est l'une des notations les plus explicites pour les ensembles de bases :
- le trait d'union (-) sépare la référence aux orbitales centrales et de valence ;
- les chiffres représentent le nombre de primitives Gaussiennes dans une fonction ;
- il y a autant de chiffres qu'il y a de fonctions pour chaque orbitale atomique (zeta $\zeta$) ;
- la lettre G fait référence à l'utilisation de fonctions Gaussiennes.

À titre d'exemple, l'ensemble de base double zêta `3-21G` utilise trois primitives Gaussiennes pour chaque orbitale centrale et un ensemble de deux plus une primitives Gaussiennes pour les orbitales de valence. Plus d'informations, d'exemples et de discussions sur les GTO peuvent être trouvés dans ce [lien](https://chem.libretexts.org/Courses/Pacific_Union_College/Quantum_Chemistry/11%3A_Computational_Quantum_Chemistry/11.02%3A_Gaussian_Basis_Sets).

Généralement, on adopte des ensembles identiques de fonctions de base pour tous les atomes d'un élément donné et les ensembles adoptés pour les différents éléments doivent avoir une précision équilibrée. Une déclaration telle que « *nous avons utilisé la base 6-31G définie dans le calcul* » signifie qu'un ensemble équilibré d'ensembles d'AO a été adopté pour tous les éléments.

L’ensemble de base étant une approximation, il est hautement souhaitable de pouvoir contrôler sa précision afin de faire des compromis entre le coût du calcul et la précision des résultats obtenus. En conséquence, les ensembles de base modernes se répartissent généralement en familles de tailles variables : les ensembles les plus petits permettent des calculs rapides mais qualitatifs, tandis que les ensembles plus grands permettent des calculs quantitatifs au prix de plus de temps de calcul.

La fonction d'état pour une molécule est une approximation lors de l'utilisation d'un ensemble de base finie,
$$
\Phi_{\text{approx}} = \sum_{i=0}^N c_i |i \rangle
$$

En vertu du principe variationnel, en considérant que $\Phi_{\text{approx}}$ est normalisée, l'énergie approximée est supérieure ou égale à la véritable énergie de l'état fondamentale,
$$
\langle \Phi_{\text{approx}} | \hat{H} | \Phi_{\text{approx}} \rangle = \epsilon_{\text{approx}} \geq  \epsilon_{\text{true}}
$$

A mesure que l'ensemble de base s'agrandit, l'approximation résultante  se rapproche de la véritable fonction d'état,
$$
\Phi_{\text{approx}} = \sum_{i=0}^{\infty} c_i |i \rangle \approx \Psi_{\text{true}}
$$

Cependant, augmenter la taille de l'ensemble de base a pour effet d'augmenter le nombre de paramètres à optimiser, étendant ainsi les ressources de calculs requises dans l'espace (espace de stockage) et en temps (nombre d'opérations). Il est souvent judicieux d'équilibrer la taille de l'ensemble de base pour optimiser le temps de résolution par rapport à la précision cible ($|\epsilon_{\text{approx}} -  \epsilon_{\text{true}}|$), puisque l'augmentation de la complexité de l'ensemble de base produit des rendements décroissants en termes de gain de précision.

 De plus en plus de bases sont disponibles pour la communauté, et le choix d'un ensemble de bases peut s'avérer une tâche difficile. La théorie stipule qu'une base infinie est le moyen d'obtenir de vrais calculs des fonctions d'état. En pratique, il n'est pas possible d'effectuer les calculs avec des ensembles de bases infinis et d'obtenir des vraies fonction d'état. On peut cependant sélectionner des ensembles de bases finis et adapter nos approches aux cas d'utilisation. En règle générale, 
* un double zêta (par exemple `6-31G` ou `cc-pvdz`) est la taille de base minimale requise pour obtenir des énergies semi-quantitatives;
* pour les molécules présentant une différence d'électronégativité élevée entre les atomes, l'ajout de fonctions de polarisation (par exemple `6-31G**` ou `6-31G(d,p)`) est recommandé;
* les fonctions diffuses (par exemple `6-31G+(d,p)`) doivent être incluses lorsqu'il s'agit d'anions. Les fonctions diffuses sont des fonctions avec de petits coefficients (petit $\alpha$ dans $e^{-\alpha r^2}$) qui peuvent s'adapter à la densité électronique loin du noyau;
* les ensembles de bases des potentiels de noyau effectif (Effective Core Potential, ECP) sont nécessaires lorsqu'il s'agit d'atomes plus lourds avec des effets relativistes et des effets non triviaux (par exemple, l'atome d'iode est souvent modélisé dans l'ensemble de base `LANDL2DZ`).

**Tableau** : Catégories d'ensemble de base.
| Catégorie | Caractère | Exemple |
| -------- | --------- | ------- |
| Minimale| Fonction de base unique par orbitale atomique occupée, la taille des éléments de la deuxième rangée est $[2s1p]$ | STO-3G |
| Double-$\zeta$ | Deux fonctions de base par orbitale atomique de valence occupée, la taille des éléments de la deuxième rangée est $[3s2p]$ | 6-31G |
| Triple-$\zeta$ | Trois fonctions de base par orbitale atomique de valence occupée, pour les éléments de deuxième rangée $[4s3p]$ | 6-311G |
| Polarisation | Fonctions de base avec un moment cinétique plus élevé ; important pour la liaison chimique ; la taille des éléments de la deuxième ligne est $[3s2p1d]$ | cc-pVDZ |
| Diffusion | Fonctions de base avec de petits exposants ; important pour les états excités ; les tailles des éléments de la deuxième rangée sont $[4s3p]$ pour 6-31+G et $[4s3p2d]$ pour aug-cc-pVDZ | 6-31+G, aug-cc-pVDZ |


### Ensembles de base de karlsruhe

 Contrairement aux ensembles de bases traditionnels, les familles d'ensembles de bases modernes permettent une approche rentable de la limite complète de l'ensemble de bases, auquel cas l'erreur dans l'ensemble de base à un électron n'affecte plus le calcul. Notez que d'autres types d'ensembles de bases que les gaussiennes peuvent également être utilisés pour la chimie quantique.

La famille Karlsruhe `def2` d'ensembles de bases gaussiennes, qui constituent un bon choix complet pour la chimie quantique car ils sont disponibles pour l'ensemble du tableau périodique jusqu'au radon (Z = 86). Les ensembles Karlsruhe def2 sont disponibles en trois niveaux de précision. 

* Les ensembles de bases à valence divisée (SV) constituent le plus petit ensemble de bases raisonnable pour les applications générales. La base `def2-SVP` est un ensemble de base SV avec des fonctions de polarisation (P) et est de taille similaire au `6-31G**` également connu sous le nom d'ensemble de base `6-31G(d,p)`. Comme le `6-31G**`, l'ensemble `def2-SVP` peut également être utilisé sans fonctions de polarisation sur les atomes d'hydrogène ; cette base est appelée `def2-SV(P)`, elle est plus petite que la base `6-31G*`, et elle est souvent utile pour des calculs qualitatifs/semi-quantitatifs rapides. 

* Pour des calculs plus quantitatifs, la série def2 contient également un ensemble de polarisations de valence triple ζ (`def2-TZVP`) ainsi qu'un ensemble de polarisations de valence quadruple ζ (`def2-QZVP`), qui suffisent généralement pour atteindre la limite complète de l'ensemble de base en HF. et calculs DFT. * Les calculs aux niveaux théoriques post-HF, nécessitent, cependant, des ensembles de bases plus larges avec des fonctions de polarisation supplémentaires ; les ensembles de base `def2-TZVPP` et `def2-QZVPP` existent à cet effet. 

* Les fonctions diffuses (D) sont nécessaires à la description correcte des anions ainsi qu'à la modélisation, de la polarisabilité électrique par exemple. Des ensembles sont à cet effet disponibles à tous les niveaux de précision (`def2-SVPD`, `def2-TZVPD`, `def2-TZVPPD`, `def2-QZVPD`, `def2-QZVPPD`).

Ci-dessous, des exemples d’ensembles de base populaires utilisés en chimie quantique. Une liste complète des ensembles de bases disponibles pris en charge dans [PySCF](https://pyscf.org/) est disponible sur ce [lien](https://pyscf.org/_modules/pyscf/gto/basis.html).


In [ ]:
basis_sets = [
    "STO-3G",           # Simple zeta, minimal basis
    "3-21G",            # Double zeta with 3 Gaussian primitives
    "6-31G",            # Double zeta with 6 Gaussian primitives
    "6-31G*",           # Double zeta with 6 Gaussian primitives
    "6-31G(d,p)",       # Polarization functions (+ 5 d-orbitals for all atoms except H, +3 p-orbitals for H atoms) added
    "6-311G(d,p)",      # Triple zeta with polarization functions
    "6-311+G(d,p)",     # Triple zeta with polarization functions and diffuse functions
    "cc-pvqz",          # Quadruple zeta
    "cc-pv5z",          # Quintuple zeta
    "def2-SVP",         # Double zeta with polarization functions
    "def2-SVPD",         # Double zeta with polarization functions and diffuse functions
    "def2-TZVP",        # Triple zeta with polarization functions
    "def2-TZVPD",       # Triple zeta with polarization functions and diffuse functions
    "def2-TZVPP",       # Triple zeta with polarization functions and diffuse functions
    "def2-TZVPPD",      # Triple zeta with polarization functions, diffuse functions, and d functions
    "def2-QZVP",        # Quadruple zeta with polarization functions
    "def2-QZVPP",       # Quadruple zeta with polarization functions and diffuse functions
    "def2-QZVPPD",      # Quadruple zeta with polarization functions, diffuse functions, and d functions
    "def2-QZVPD",       # Quadruple zeta with polarization functions and diffuse functions
    "def2-QZVPPD",      # Quadruple zeta with polarization functions, diffuse functions, and d functions
]

### <a id='toc1_4_5_'></a>[Effet de l'ensemble de base sur les calculs d'énergie totale et le temps de calculs](#toc0_)

In [ ]:
import time

mf_energies = list()
mf_times = list()
nb_prim = list()

# Perform a Mean-Field calculation for each basis set
for bs in basis_sets:

    # Measure execution time
    start = time.time()
    DWat_mol.basis = bs
    DWat_mol.build()

    mf = scf.RHF(DWat_mol)
    mf.kernel()
    end = time.time()

    nb_prim.append(DWat_mol.npgto_nr())
    mf_energies.append(mf.e_tot)
    mf_times.append(end-start)


In [ ]:
# Create the results dataframe
import pandas as pd

df_HF = pd.DataFrame({"Basis":basis_sets, 
                      'Nb of GTO primitives':nb_prim, 
                      'Total energy':mf_energies,
                      "Time":mf_times})

df_HF


* Illustration avec `matplotlib.pyplot` qui est par défaut statique, mais que nous allons rendre interactif avec la commande magique `%matplotlib notebook`.

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

# Create the matplotlib figure
fig, ax = plt.subplots(figsize=(8,5))

# Plot the energies.
ax.set_xticks(range(len(basis_sets)), basis_sets, rotation=90)
ax.set_xlabel("Basis set")
ax.set_ylabel("Energy(Hartree)", color="b")
ax.scatter(range(len(basis_sets)), mf_energies, marker="o", s=50, color="b")

# Plot the time to solution
ax_time = ax.twinx()
ax_time.scatter(range(len(basis_sets)), mf_times, marker="s", s=50, color="r")
ax_time.set_ylabel("Time to solution(s)", color="r", rotation=270, va="bottom")

# Show the graph
plt.tick_params(axis="both", direction="in")
plt.show()

* Illustration avec `plotly.express` qui est interactif par défaut.

In [ ]:
import plotly.express as px

# Create the figure
fig = px.scatter()

# Convert basis_sets to a list
basis_indices = list(range(len(basis_sets)))

# Plot the energies
fig.add_scatter(x=basis_indices, y=mf_energies, mode="markers",\
                marker=dict(symbol="circle", size=10, color="blue"), name="Energy (Hartree)")

# Plot the time to solution
fig.add_scatter(x=basis_indices, y=mf_times, mode="markers",\
                marker=dict(symbol="square", size=10, color="red"), name="Time to solution (s)", yaxis="y2")

# Set the x-axis labels and tick labels
fig.update_layout(xaxis=dict(tickmode="array", tickvals=basis_indices, ticktext=basis_sets, tickangle=90))

# Set the y-axis labels and colors
fig.update_layout(yaxis=dict(title="Energy(Hartree)", color="blue"),\
                  yaxis2=dict(title="Time to solution (s)", color="red", overlaying="y",\
                              side="right"))

# Show the graph
fig.show()

Le tableau précédent et cette figure suggèrent `def2-TZVPP` comme le bon ensemble de bases pour cette molécule.